# Explore here

In [37]:
# Your code here
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import *
from imblearn.metrics import specificity_score
from sklearn.model_selection import RandomizedSearchCV
import joblib
from sklearn.ensemble import RandomForestClassifier



In [38]:
df=pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv",sep=",")

In [39]:
df

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0
...,...,...,...
886,com.rovio.angrybirds,loved it i loooooooooooooovvved it because it...,1
887,com.rovio.angrybirds,all time legendary game the birthday party le...,1
888,com.rovio.angrybirds,ads are way to heavy listen to the bad review...,0
889,com.rovio.angrybirds,fun works perfectly well. ads aren't as annoy...,1


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [41]:
df.drop(columns=['package_name'],axis=1,inplace=True)
df["review"][0]

" privacy at least put some option appear offline. i mean for some people like me it's a big pressure to be seen online like you need to response on every message or else you be called seenzone only. if only i wanna do on facebook is to read on my newsfeed and just wanna response on message i want to. pls reconsidered my review. i tried to turn off chat but still can see me as online."

In [42]:
df["review"] = df["review"].str.strip().str.lower()
df["review"]

0      privacy at least put some option appear offlin...
1      messenger issues ever since the last update, i...
2      profile any time my wife or anybody has more t...
3      the new features suck for those of us who don'...
4      forced reload on uploading pic on replying com...
                             ...                        
886    loved it i loooooooooooooovvved it because it ...
887    all time legendary game the birthday party lev...
888    ads are way to heavy listen to the bad reviews...
889    fun works perfectly well. ads aren't as annoyi...
890    they're everywhere i see angry birds everywher...
Name: review, Length: 891, dtype: object

In [43]:
X=df['review']
y=df['polarity']
X

0      privacy at least put some option appear offlin...
1      messenger issues ever since the last update, i...
2      profile any time my wife or anybody has more t...
3      the new features suck for those of us who don'...
4      forced reload on uploading pic on replying com...
                             ...                        
886    loved it i loooooooooooooovvved it because it ...
887    all time legendary game the birthday party lev...
888    ads are way to heavy listen to the bad reviews...
889    fun works perfectly well. ads aren't as annoyi...
890    they're everywhere i see angry birds everywher...
Name: review, Length: 891, dtype: object

In [44]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train

331    just did the latest update on viber and yet ag...
733    keeps crashing it only works well in extreme d...
382    the fail boat has arrived the 6.0 version is t...
704    superfast, just as i remember it ! opera mini ...
813    installed and immediately deleted this crap i ...
                             ...                        
106    why can't i share my achievements? recently di...
270    beta is the best version of the chrome browser...
860    great little game. this is a great little game...
435    keeps crashing ever since i started using it m...
102    even though i am loving the new update, but th...
Name: review, Length: 712, dtype: object

In [45]:
vec_model = CountVectorizer(stop_words="english")
X_train = vec_model.fit_transform(X_train).toarray()
X_test = vec_model.transform(X_test).toarray()
X_train


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [46]:
gaus=GaussianNB().fit(X_train,y_train)
gaus.fit(X_train,y_train)


GaussianNB()

In [47]:
y_pred_train_gaussian=gaus.predict(X_train)
y_pred_test_gaussian=gaus.predict(X_test)


In [48]:
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)

    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)

    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])

    return metrics_df

In [49]:
get_metrics(y_train,y_test,y_pred_train_gaussian,y_pred_test_gaussian)



,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.985955,0.980695,0.989083,0.962121,1.000000,0.978166
Test,0.804469,0.653465,0.751797,0.687500,0.622642,0.880952
Diferencia,0.181486,0.327230,0.237286,0.274621,0.377358,0.097214


In [50]:
multi=MultinomialNB().fit(X_train,y_train)
multi.fit(X_train,y_train)


MultinomialNB()

In [51]:
y_pred_train_multi=multi.predict(X_train)
y_pred_test_multi=multi.predict(X_test)

In [52]:
get_metrics(y_train,y_test,y_pred_train_multi,y_pred_test_multi)


,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.960674,0.944223,0.954527,0.955645,0.933071,0.975983
Test,0.815642,0.659794,0.754268,0.727273,0.603774,0.904762
Diferencia,0.145032,0.284429,0.200259,0.228372,0.329297,0.071221


In [53]:
ber=BernoulliNB().fit(X_train,y_train)
ber.fit(X_train,y_train)

BernoulliNB()

In [54]:
y_pred_train_ber=ber.predict(X_train)
y_pred_test_ber=ber.predict(X_test)

In [55]:
get_metrics(y_train,y_test,y_pred_train_ber,y_pred_test_ber)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.919944,0.875817,0.891302,0.980488,0.791339,0.991266
Test,0.770950,0.506024,0.662399,0.700000,0.396226,0.928571
Diferencia,0.148994,0.369793,0.228904,0.280488,0.395112,0.062695


De entre todos los modelos me quedaré con el MultinomialNB ya que se está trabajando con datos discretos obtenido por CountVectorized y la variable objectivo es categórica 

In [56]:
hp_params={
    'alpha': np.linspace(0.01,10.0,100),
    'fit_prior':[True,False]
}

random_search=RandomizedSearchCV(multi,hp_params,n_iter=100,scoring="accuracy",cv=5,random_state=42)
random_search.fit(X_train,y_train)


RandomizedSearchCV(cv=5, estimator=MultinomialNB(), n_iter=100,
                   param_distributions={'alpha': array([ 0.01      ,  0.11090909,  0.21181818,  0.31272727,  0.41363636,
        0.51454545,  0.61545455,  0.71636364,  0.81727273,  0.91818182,
        1.01909091,  1.12      ,  1.22090909,  1.32181818,  1.42272727,
        1.52363636,  1.62454545,  1.72545455,  1.82636364,  1.92727273,
        2.02818182,  2.12909091,  2.23      ,  2.33090909,  2.43181...
        7.07363636,  7.17454545,  7.27545455,  7.37636364,  7.47727273,
        7.57818182,  7.67909091,  7.78      ,  7.88090909,  7.98181818,
        8.08272727,  8.18363636,  8.28454545,  8.38545455,  8.48636364,
        8.58727273,  8.68818182,  8.78909091,  8.89      ,  8.99090909,
        9.09181818,  9.19272727,  9.29363636,  9.39454545,  9.49545455,
        9.59636364,  9.69727273,  9.79818182,  9.89909091, 10.        ]),
                                        'fit_prior': [True, False]},
                   random_state=42, scoring='accuracy')

In [57]:
random_search.best_estimator_

MultinomialNB(alpha=np.float64(2.028181818181818), fit_prior=False)

In [58]:
Multinomial_optimazed=MultinomialNB(alpha=2.028181818181818,fit_prior=False)
Multinomial_optimazed.fit(X_train,y_train)

MultinomialNB(alpha=2.028181818181818, fit_prior=False)

In [59]:
y_pred_train_multi_optimazed=Multinomial_optimazed.predict(X_train)
y_pred_test_multi_optimazed=Multinomial_optimazed.predict(X_test)

In [60]:
get_metrics(y_train,y_test,y_pred_train_multi_optimazed,y_pred_test_multi_optimazed)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.948034,0.926148,0.940317,0.939271,0.913386,0.967249
Test,0.821229,0.680000,0.769167,0.723404,0.641509,0.896825
Diferencia,0.126805,0.246148,0.171150,0.215867,0.271876,0.070424


Se ha disminuido en un pequeña cantidad el overfitting en el conjunto de entrenamiento aunque no he hecho una búsqueda de hiperparametros tan exhaustiva.

In [61]:
get_metrics(y_train,y_test,y_pred_train_multi,y_pred_test_multi)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.960674,0.944223,0.954527,0.955645,0.933071,0.975983
Test,0.815642,0.659794,0.754268,0.727273,0.603774,0.904762
Diferencia,0.145032,0.284429,0.200259,0.228372,0.329297,0.071221


In [62]:
joblib.dump(Multinomial_optimazed,'multinomialnb_alpha2.028181818181818_fit_priorFalse.pk1')


['multinomialnb_alpha2.028181818181818_fit_priorFalse.pk1']

In [63]:
rand_Forest= RandomForestClassifier(n_estimators=100,max_depth=15,min_samples_leaf=5,min_samples_split=10,max_features='sqrt',bootstrap=True,random_state=42,n_jobs=-1)

In [64]:
rand_Forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=15, min_samples_leaf=5, min_samples_split=10,
                       n_jobs=-1, random_state=42)

In [65]:
y_pred_train=rand_Forest.predict(X_train)
y_pred_test=rand_Forest.predict(X_test)

In [66]:
get_metrics(y_train,y_test,y_pred_train,y_pred_test)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.727528,0.389937,0.619864,0.96875,0.244094,0.995633
Test,0.715084,0.163934,0.535265,0.62500,0.094340,0.976190
Diferencia,0.012444,0.226003,0.084599,0.34375,0.149755,0.019443


In [67]:
get_metrics(y_train,y_test,y_pred_train_multi_optimazed,y_pred_test_multi_optimazed)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.948034,0.926148,0.940317,0.939271,0.913386,0.967249
Test,0.821229,0.680000,0.769167,0.723404,0.641509,0.896825
Diferencia,0.126805,0.246148,0.171150,0.215867,0.271876,0.070424


El modelo multinomialnb optimizado tiene un mejor desempeño tanto en entrenamiento como en los datos de testeo que el modelo de randomForest. Aunque el modelo de multinomial presenta un mayor overfitting tienen un mejor desempeño en datos nunca antes vistos en comparación con el modelo de arboles

In [68]:
joblib.dump(rand_Forest,'randomForest.pk1')


['randomForest.pk1']